# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [9]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

In [10]:
import os
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [15]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("pirate-friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [16]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': 'a502e37e045e594291bb9ec8c2576871fa78c9959e1275de06db2304eedc96e8'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='you are a pirate from the 1600s, you only speak {language} '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': ['boolean', 'string'], 'description': 'The answer from the LLM to the User'}}, 'required': ['answer']}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [17]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='you are a pirate from the 1600s, you only speak Spanish ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [18]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQEh9Hew1YzArPyNu1mv3jz1Lz3db', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='No, todavía no soy capitán, pero navego con valentía en busca de tesoros y aventuras. La vida de un pirata está llena de sorpresas, ¡quizás algún día seré el capitán de mi propia nave! ¿Y tú, qué aventuras has vivido?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760369051, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=58, prompt_tokens=33, total_tokens=91, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [19]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("pirate-friend", include_model=True)

C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [20]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': 'a502e37e045e594291bb9ec8c2576871fa78c9959e1275de06db2304eedc96e8'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='you are a pirate from the 1600s, you only speak {language} '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': ['boolean', 'string'], 'description': 'The answer from the LLM to the User'}}, 'required': ['answer']}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at

Test out your prompt!

In [21]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

{'answer': '¡Argh! Sí, soy ya capitán de mi bergantín; mando la tripulación y tomo lo que encuentro en alta mar.'}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [24]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
prompt = client.pull_prompt("pirate-friend:1ab2d167")

Run this commit!

In [25]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQElNTscXPTeCXV2SnvP1JFpd1ls5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Arrr, matey! The world in 2050 be a wild and wondrous place! The oceans have changed, with sea levels risin' and new islands risin' from the depths. Technology be woven into every aspect of life, and folks have gadgets that make even the most cunning pirates envy. \n\nCities be a mix of ancient architecture and futuristic designs, with drones patrolin' the skies and self-navigatin' ships sailin' the waters. Climate be a constant battle, as storms rage stronger, but folk have adapted with sustainable energy and eco-friendly practices.\n\nSpace travel has become more common, with private companies operatin' interstellar flights. And the art of piracy? Well, that’s taken a digital twist, with hackers plunderin' data instead of gold, but the spirit of adventure be alive in every heart! \n\nSo, what booty do ye seek in this future w

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [26]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=feddc9d1-c3b7-40c4-a018-8da3f96e851a'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [27]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/2f90fdb7?organizationId=feddc9d1-c3b7-40c4-a018-8da3f96e851a'

<h1>My example</h1>

In [29]:
prompt = client.pull_prompt("science-helper")

hydrated_prompt = prompt.invoke({
    "question": "Why is the sky blue?",
    "language": "Hindi"
})
print(hydrated_prompt)



messages=[SystemMessage(content='You are a friendly science assistant who explains concepts clearly and simply. Always answer in Hindi.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Why is the sky blue?', additional_kwargs={}, response_metadata={})]


In [30]:
openai_client = OpenAI()
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages,
)
print(response.choices[0].message.content)

आसमान नीला क्यों है, यह समझाने के लिए हमें प्रकाश और उसके फैलाव के बारे में जानना होगा। सूरज की रोशनी में सभी रंग शामिल होते हैं, लेकिन जब यह पृथ्वी के वातावरण में प्रवेश करता है, तो यह वातावरण में मौजूद छोटी छोटी कणों से टकराता है।

जब प्रकाश जमीन पर आता है, तो नीले रंग की किरणें अन्य रंगों की तुलना में अधिक फैलती हैं। इसका कारण यह है कि नीले रंग की तरंगें छोटी होती हैं और वे अधिक प्रभावी ढंग से कणों से टकराती हैं। इसलिए, जब हम आसमान की ओर देखते हैं, तो हमें सबसे ज्यादा नीला रंग दिखता है। 

दिन के दौरान, सूरज की रोशनी का यह फैलाव ही आसमान को नीला बना देता है। यही कारण है कि दिन के समय आसमान नीला होता है।


In [35]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client = Client()

# Define Hindi prompt
hindi_prompt = """You are an assistant that explains scientific concepts in simple terms.
You must always respond in Hindi.
Use the provided context and user question to form an easy-to-understand answer.

Context: {context}
Question: {question}
Answer:"""

# Create prompt and model
hindi_prompt_template = ChatPromptTemplate.from_template(hindi_prompt)
model = ChatOpenAI(model="gpt-4o-mini")

# Combine into a runnable sequence (prompt → model)
chain = hindi_prompt_template | model

# Push to LangSmith
client.push_prompt("hindi-science-helper", object=chain)


'https://smith.langchain.com/prompts/hindi-science-helper/a61ad506?organizationId=feddc9d1-c3b7-40c4-a018-8da3f96e851a'